## Introduction 

**Background explanation only -- delete before submitting the assignment**

Boolean retrieval in one markdown cell

The problem:  given a *search query* and a *document* answer whether or not a document *satisfies* the query.

Equivalent:  given a *search query* and a *set of documents* return the subset of documents that  *satisfy* the query.

Satisfaction is either true or false.  (So no notion of relevance or satisfying partially or more.)

A query is a boolean combination of *term matches*:
```
T1
(NOT T1)
(T1 OR T2)
(T1 AND T2)
```

where T1 is a term, i.e. a word, i.e. a string of characters, and a document 
D
is a set of terms, and the query is satisfied if and only if T1 appears in D

Exact string match is a good start, but too strong
* Case sensitive?  "Hello" should probably match "hello".  But should "Apple" match "apple"?
* Punctuation?   "Hello" should probably match "Hello!". But should "second-best" be considered one term or two?
* (Lots of others -- this is a big topic, but not for today!)


------------------------------

###  Assignment 0 -- TCSS 5910

In this assignment you are going to read in some text files containing movie reviews and index them so you can perform boolean AND/OR queries.

The directory **data** has one file per review.   The name of the file will act as the document ID, so for example the file **data/Dog4.txt** will have the Document ID **Dog4**.  The file contains the review text.

The query operators will be 
* **query(term)** -- Term is a string, return a set of the IDs of all documents that contain it
* **and_query(arg1, arg2)** -- The arguments are either strings or sets of document IDs.  Return a set of the IDs of the documents at the *intersection* of the two arguments
* **or_query(arg1, arg2)** -- The arguments are either strings or sets of document IDs.  Return a set of the IDs of the documents at the *union* of the two arguments

The more complicated definition of **and_query** and **or_query** is because we want to nest them.  For example:
<pre>or_query('hello', 'world')</pre> and <pre>or_query('hello', or_query('world', 'earth'))</pre> both need to work.

At the end of the exercise we want to see this:
<pre>
#  Query on a single term gives back a set of document IDs
print(query('movie'))

#  And query
print(and_query('yellow', 'lab'))

# Nested query
print(and_query(and_query('salome', or_query('good', 'excellent')), 'opera'))

# Too bad that or_query needs exactly two arguments!
print(or_query(or_query('good', 'excellent'), or_query('liked', 'loved')))
</pre>
<pre>
{'Dog4', 'Joe1', 'Dog3', 'Dog2', 'Dog1', 'Joe2'}
{'Dog1', 'Dog2'}
{'Salome5'}
{'DooWop2', 'Dog4', 'Joe1', 'Joe2', 'Salome5', 'DooWop1'}
</pre>

---------------------------------------------------------


#### Step 1:  Read in the review files, and for each get the doc ID and review text

In [ ]:
import os
DATA_DIR = 'data'

#  Iterate over all the files in the data directory.  For each:
#    -- extract its file basename as the docID
#    -- read the file as a single string

#  Return a dictionary of {docID: review_text} pairs, one for each file in the data directory
def get_IDs_and_reviews():
    dict = {}
    # Iterate over the file names in DATA_DIR -- for each, read the review body;
    #  add the docID and the review body to the dictionary
    for filename in os.listdir(DATA_DIR):
        dict[getDocID(filename)] = readFile(filename)
    return dict

#  Return the name of the file in this path, without the directories and 
#  without the extension.  For example,   "data/Dog1.txt"  ->  "Dog1"
#
#  Hint: look at os.path.basename and os.path.splitext
def getDocID(path):
    # YOUR CODE HERE

# The filename is something like "Dog1.txt" -- open that file relative to DATA_DIR, 
#  read its contents as a string, and return that string

# Hint.  the primitive open(filename) opens a file;  A file has a read() operation
# that returns its contents as a string
def readFile(filename):
    #  YOUR CODE HERE


In [ ]:
#  Bind a variable for later use, and make sure it has the expected contents
ids_and_reviews = get_IDs_and_reviews()
print(len(ids_and_reviews))
print(ids_and_reviews)

Here is some heavily edited output.  Notice that the reviews are pretty messy, they have newlines, HTML tags, mixed capitalization, etc.

<pre>
13
{'Dog1': 'You don't have to own a Yellow Lab to absolutley love this movie, \n but ...',  
 'Dog2': '... mesmerized by this movie. &lt;em&gt;The adventures of a lost boy and dog&lt;/em&gt;....',
   ...
 'DooWop1':  'Excellent, excellent performers.  Excellent video and audio quality....', 
 'Salome5':  'I saw this production on TV many years ago and thought then that it was really ...'
}
</pre>

-----------------------------------------------------

####  Step 2: Cleaning up the review text

Next we want to do some basic cleanup:  tokenize the input into individual words, get rid of special characters, 
get rid of HTML tags etc.

Result will be the same dictionary as before, except the review text will be a list of strings (we call the 'tokens'), not a single string

In [ ]:
from nltk.tokenize import word_tokenize

#  This should return a list of strings, every string should be all lower-case letters

# Hint:  
#    the word_tokenize function in nltk.tokenize will split the string into tokens
#    the string method lower() converts a string to all lower case
#    the string method isalpha() returns true if the string is all letters 
def clean_review(review):
    #  YOUR CODE HERE

# Notice this function side-effects its input arguments
def clean_reviews(id_to_reviews):
    for docID, review in id_to_reviews.items():
        id_to_reviews[docID] = clean_review(review)


In [ ]:
clean_reviews(ids_and_reviews)
print(ids_and_reviews)

Notice that all special characters have been removed, the words are all letters, and all in lower case.
Removing all special characters might be too extreme in a real system, for example our code will remove *don't* and *well-made* -- but it's simple and good enough for now.

<pre>
{'Dog1': ['you', 'do', 'have', 'to', 'own', 'a', 'yellow', 'lab', ...],
 'Dog3': ['you', 'do', 'have', 'to', 'be', 'a', 'kid', 'to', 'enjoy', ...],
 'Joe1': ['i', 'not', 'a', 'tim', 'allen', 'fan', 'i', 'had', 'only', 'watched', ...],
  ... 
 'Salome5': ['i', 'saw', 'this', 'production', 'on', 'tv', 'many', 'years', 'ago',...]}
</pre>

--------------------

####  Step 3:  Build the document-to-term dictionaries

In this step we convert the list of words to a dictionary:  the key is a string (term/token/word) and the value is a set containing exactly one docID.

For example in the output above we should get a dictionary that looks like this, partially
<pre>
{ 'you': {'Dog1'},
  'do':  {'Dog1},
  'have': {'Dog1} ...
 }
</pre>

This is the dictionary only for the first document/review -- we will have a different dictionary for each of the 13 reviews.

The dictionary's value is always a set with one element, and the one element will be the same for each review -- the docID for that review.  It has that format so next we can merge all of the 13 dictionaries to get a single dictionary of a term to the IDs of all the documents that contain that term.

In [ ]:
#  Input is a dictionary docID -> termList
#  Output is a list of dictionaries, one per review, each of the form   term -> {docID}

# Hint:
#  if the variable docID is a string, then {docID} creates a set containing just that string
#  To iterate over the key/value pairs in a dictionary, use   
#      for docID, terms in docs_to_terms.items():

def build_term_to_doc_dictionaries(docs_to_terms):
    dd = []
    #  YOUR CODE HERE
    return dd

In [ ]:
term_to_doc = build_term_to_doc_dictionaries(ids_and_reviews)

This is a list of 13 dictionaries.  Each dictionary has keys which are terms, and a value which is a set containing exactly one document ID

<pre>
[{'you': {'Dog1'},
  'do': {'Dog1'},
  'have': {'Dog1'},
  ...
  }, 
 {'as': {'Dog2'},
  'owners': {'Dog2'},

  'wildlife': {'Dog4'},
  'wolves': {'Dog4'},
  'wild': {'Dog4'},
  'cats': {'Dog4'},
  ...
  },
  ...
 {'excellent': {'DooWop1'},
  'performers': {'DooWop1'},
  ...
  },
  ...
  ]
 </pre>
 

------------------------------------

#### Step 4:  Merge Term to Doc ID 

This is the last preprocessing step -- we need to merge these 13 term to document dictionaries into a single dictionary that looks like:   
<pre>
term: {docID1, docID2, ...} 
</pre>
in other words, for every term that appears in *any* document, a set/list of the documents containing it.

This step is a little tricky because if I want to merge two dictionaries *d1* and *d2* there will be some terms that appear in *d1* but not *d2*, some that appear in *d2* but not *d1*, and some that appear in both.  

In [ ]:
# Create a new dictionary that has keys that are the union of d1's and d2's keys 
# and whose value for each key is a set which is the union of d1's and d2's values
# for that key.   If a key is not in d1's or d2's dictionary, its value for that 
# key should be interpreted as the empty set.

# Hint -- start by getting all the terms --     
#   allTerms = set(d1.keys()) | set(d2.keys())

def merge_two(d1, d2):
 # YOUR CODE HERE

# Merge a list of indexes sequentially
def merge_indexes(indexes):
    dout = {}
    for d in indexes:
        dout = merge_two(dout, d)
    return dout

In [ ]:
term_to_docs = merge_indexes(term_to_doc)

In [ ]:
print(term_to_docs)

This is edited output, so yours won't be in this order, though the values should be the same.

This is the inverted index we will query against.   Note that there are some useless words like 'with' and 'while' that probably should be excluded.  These are known as *stop words* 

<pre>
{ 'comforts': {'Dog4'}, 
  'about': {'Salome1'}, ...
  'first': {'DooWop2', 'Joe2', 'Salome2'}, 
  'with': {'DooWop2', 'Dog4', 'Joe1', 'Salome5', 'Salome2', 'Dog3', 'Salome1', 'Joe2', 'Salome4', 'Salome3'}, 
  'thought': {'DooWop2', 'Salome5'}, 
  'buy': {'DooWop1', 'Dog2', 'Salome1'}, 
  'while': {'DooWop2', 'Dog4', 'Dog1', 'Salome1', 'Salome4'}, 
  ...
}
</pre>

#### Step 5:  Queries

Now it's easy to do queries on the index

* for a simple query (single term) just look up the term in the index and retrieve its doc IDs
* for an AND query, take the intersection of the doc IDs
* for an OR query, take the union of the doc IDs

The complication is that in order to do nesting, *and_query* and *or_query* need to accept either a term (string) or a set of docIDs as input.    


In [ ]:
# Remember that we are assuming a global variable here.  Messy! 
INVERTED_INDEX = term_to_docs

# Access INVERTED_INDEX to get the docIDs for this term. 
# If the term is not in the index, return empty set
def query(term):
    # YOUR CODE HERE

# If either or both arguments are strings, first call query() to get a set of doc IDs.
#  Once both arguments are sets, just return the intersection of the sets.

def and_query(term1, term2):
    #  YOUR CODE HERE

def or_query(term1, term2):
    #  YOUR CODE HERE


In [ ]:
#  Query on a single term gives back a set of document IDs
print(query('movie'))

#  And query
print(and_query('yellow', 'lab'))

# Nested query
print(and_query(and_query('salome', or_query('good', 'excellent')), 'opera'))

# Too bad that or_query needs exactly two arguments!
print(or_query(or_query('good', 'excellent'), or_query('liked', 'loved')))

<pre>
{'Dog4', 'Dog3', 'Dog2', 'Dog1', 'Joe1', 'Joe2'}
{'Dog1', 'Dog2'}
{'Salome5'}
{'DooWop2', 'Dog4', 'Joe1', 'Joe2', 'Salome5', 'DooWop1'}
</pre>